In [61]:
#import packages and create date

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import shutil
import difflib
from pulp import *
import openpyxl
import glob

from datetime import *
today = date.today()
today = today.strftime("%m.%d.20%y")
today

time = "9pm_"
# import datetime
# today = date.today()
# today = today - datetime.timedelta(days=1)
# today = today.strftime("%m.%d.20%y")
# today

In [62]:
## FanDuel Player Pool



contest = "PlayerPool_"
os.chdir(r"C:\Users\vchang\Downloads")

downloads = r'C:/Users/vchang/Downloads'
contest_data_loc = r'C:/Users/vchang/Documents/Fantasy Bball/DFS/FD_Data/PlayerPool'

fd = glob.glob('*-players-list.csv')[0]
os.rename(fd, "FDSalaries_"+ contest + str(today)+".csv")

shutil.move(downloads + "/"+ "FDSalaries_"+ contest + str(today)+".csv",
           contest_data_loc + "/"+ "FDSalaries_"+ contest + str(today)+".csv")

os.chdir(r"C:/Users/vchang/Documents/Fantasy Bball/DFS/FD_Data/PlayerPool")
fanduel_file = pd.read_csv(r"C:/Users/vchang/Documents/Fantasy Bball/DFS/FD_Data/PlayerPool/FDSalaries_"+ contest + str(today)+".csv")
fanduel_file.to_excel(r"C:/Users/vchang/Documents/Fantasy Bball/DFS/FD_Data/PlayerPool/FDSalaries_"+ contest + str(today)+".xlsx", index=False)
os.remove("FDSalaries_"+ contest + str(today)+".csv")


In [63]:
## Uploads
os.chdir(r"C:\Users\vchang\Downloads")

fanduel_upload = r"C:\Users\vchang\Documents\Fantasy Bball\DFS\Upload\FanDuel"

fd_upload = glob.glob('*upload-template.csv')[0]
os.rename(fd_upload, "FD_Upload_"+ contest + str(today)+".csv")

shutil.move(downloads + "/"+ "FD_Upload_"+ contest + str(today)+".csv",
           fanduel_upload + "/"+ "FD_Upload_"+ contest + str(today)+".csv")

'C:\\Users\\vchang\\Documents\\Fantasy Bball\\DFS\\Upload\\FanDuel/FD_Upload_PlayerPool_03.16.2022.csv'

In [64]:
## ETR Daily

# changes the working directory to your downloads folder
os.chdir(r"C:\Users\vchang\Downloads")

downloads = r'C:/Users/vchang/Downloads'
data_loc = r'C:/Users/vchang/Documents/Fantasy Bball/DFS/ETR_Data/FanDuel'
daily_file = "FD NBA Projections.csv"

shutil.move(downloads + "/"+ daily_file,
           data_loc + "/"+ daily_file)

os.chdir(data_loc)

df = pd.read_csv(daily_file)

df.to_excel("ETR_Daily_"+str(today)+".xlsx", index=False)
os.remove("FD NBA Projections.csv")

In [83]:
## Joins the FD and ETR Files to include FD Positions 
    ## ETR does not include G/F/UTIL

os.chdir(r"C:\Users\vchang\Documents\Fantasy Bball\DFS\PlayerPool Analysis")
etr = pd.read_excel(r'C:\Users\vchang\Documents\Fantasy Bball\DFS\ETR_Data\FanDuel\ETR_Daily_' + str(today) + ".xlsx")
fd = pd.read_excel(r'C:\Users\vchang\Documents\Fantasy Bball\DFS\FD_Data\PlayerPool\FDSalaries_PlayerPool_' + str(today) + ".xlsx")

## TO DO - check if merge on only DKNG Roster Position Works
df = pd.merge(etr, 
              fd[['Nickname','Position', 'Id']], 
              left_on='Player', 
              right_on='Nickname')

In [84]:
df.head()

,Player,Team,Opp,Minutes,FD Position,FD Salary,FD Points,FD Value,FDCeiling,FD Own,FD Slate,Nickname,Position,Id
0,Giannis Antetokounmpo,MIL,SAC,34.0,PF/SF,"11,500",57.7,7.1,67.4,29.4,MAIN,Giannis Antetokounmpo,PF/SF,73028-40199
1,Stephen Curry,GSW,BOS,36.0,PG,"10,000",43.6,-1.4,53.2,0.0,MAIN,Stephen Curry,PG,73028-9524
2,Jayson Tatum,BOS,GSW,38.0,SF/PF,"9,700",46.0,-0.6,55.8,0.2,MAIN,Jayson Tatum,SF/PF,73028-80808
3,De'Aaron Fox,SAC,MIL,38.0,PG,"9,300",42.9,-0.1,53.2,0.6,MAIN,De'Aaron Fox,PG,73028-80810
4,Pascal Siakam,TOR,LAC,38.5,PF,"8,800",41.9,1.7,50.4,1.2,MAIN,Pascal Siakam,PF,73028-60021


In [85]:
# Add 'G' Column
# Add 'F' Column
df['G Position'] = np.where(df['FD Position'].str.contains("G"), 'G', "")
df['F Position'] = np.where(df['FD Position'].str.contains("F"), 'F', "")

df['G Position'] = df['G Position'].replace('', np.nan, regex=True)
df['F Position'] = df['F Position'].replace('', np.nan, regex=True)

df['list'] = df[['FD Position', 'G Position', 'F Position']].apply(lambda x: '/'.join(x.dropna()), axis=1)

# ## Update Salary to INT 
df['FD Salary'] = df['FD Salary'].str.replace(",","").astype(float)

df['Name + ID'] = df['Id'] + ":" + df['Player']

df.head()

,Player,Team,Opp,Minutes,FD Position,FD Salary,FD Points,FD Value,FDCeiling,FD Own,FD Slate,Nickname,Position,Id,G Position,F Position,list,Name + ID
0,Giannis Antetokounmpo,MIL,SAC,34.0,PF/SF,11500.0,57.7,7.1,67.4,29.4,MAIN,Giannis Antetokounmpo,PF/SF,73028-40199,NaN,F,PF/SF/F,73028-40199:Giannis Antetokounmpo
1,Stephen Curry,GSW,BOS,36.0,PG,10000.0,43.6,-1.4,53.2,0.0,MAIN,Stephen Curry,PG,73028-9524,G,NaN,PG/G,73028-9524:Stephen Curry
2,Jayson Tatum,BOS,GSW,38.0,SF/PF,9700.0,46.0,-0.6,55.8,0.2,MAIN,Jayson Tatum,SF/PF,73028-80808,NaN,F,SF/PF/F,73028-80808:Jayson Tatum
3,De'Aaron Fox,SAC,MIL,38.0,PG,9300.0,42.9,-0.1,53.2,0.6,MAIN,De'Aaron Fox,PG,73028-80810,G,NaN,PG/G,73028-80810:De'Aaron Fox
4,Pascal Siakam,TOR,LAC,38.5,PF,8800.0,41.9,1.7,50.4,1.2,MAIN,Pascal Siakam,PF,73028-60021,NaN,F,PF/F,73028-60021:Pascal Siakam


In [86]:
df.to_csv("FD_final_PlayerPool_ETR_"+str(today)+".csv", index=False)

df = pd.read_csv(r'C:\Users\vchang\Documents\Fantasy Bball\DFS\PlayerPool Analysis\FD_final_PlayerPool_ETR_'+ str(today) + ".csv")

df.head()

,Player,Team,Opp,Minutes,FD Position,FD Salary,FD Points,FD Value,FDCeiling,FD Own,FD Slate,Nickname,Position,Id,G Position,F Position,list,Name + ID
0,Giannis Antetokounmpo,MIL,SAC,34.0,PF/SF,11500.0,57.7,7.1,67.4,29.4,MAIN,Giannis Antetokounmpo,PF/SF,73028-40199,NaN,F,PF/SF/F,73028-40199:Giannis Antetokounmpo
1,Stephen Curry,GSW,BOS,36.0,PG,10000.0,43.6,-1.4,53.2,0.0,MAIN,Stephen Curry,PG,73028-9524,G,NaN,PG/G,73028-9524:Stephen Curry
2,Jayson Tatum,BOS,GSW,38.0,SF/PF,9700.0,46.0,-0.6,55.8,0.2,MAIN,Jayson Tatum,SF/PF,73028-80808,NaN,F,SF/PF/F,73028-80808:Jayson Tatum
3,De'Aaron Fox,SAC,MIL,38.0,PG,9300.0,42.9,-0.1,53.2,0.6,MAIN,De'Aaron Fox,PG,73028-80810,G,NaN,PG/G,73028-80810:De'Aaron Fox
4,Pascal Siakam,TOR,LAC,38.5,PF,8800.0,41.9,1.7,50.4,1.2,MAIN,Pascal Siakam,PF,73028-60021,NaN,F,PF/F,73028-60021:Pascal Siakam


# Create the Constraint Problem

Goal: Maximize FD Points

- Total Players = 9
- TotalSalary <= 60000
- TotalPosition_PG = 2
- TotalPosition_SG = 2
- TotalPosition_SF = 2
- TotalPosition_PF =2 
- TotalPosition_C =1


In [87]:
## Unnests FD Positions into each Row (explode in Python) ##


## Update Salary to INT 
# df['DK Salary'] = df['DK Salary'].str.replace(",","").astype(float)

df['FD Position'] = df['FD Position'].astype(str)
df['FD Position'] = df['FD Position'].str.split('/')

df = df.explode('FD Position')

df['Position + Id'] = df['FD Position'] + "_" + df['Id']

df.head()

df.to_csv("FD_final_PlayerPool_ETR_"+str(today)+".csv", index=False)
# os.system("DKNG_final_PlayerPool_ETR_"+str(today)+".xlsx")

In [88]:
df.head()

,Player,Team,Opp,Minutes,FD Position,FD Salary,FD Points,FD Value,FDCeiling,FD Own,FD Slate,Nickname,Position,Id,G Position,F Position,list,Name + ID,Position + Id
0,Giannis Antetokounmpo,MIL,SAC,34.0,PF,11500.0,57.7,7.1,67.4,29.4,MAIN,Giannis Antetokounmpo,PF/SF,73028-40199,NaN,F,PF/SF/F,73028-40199:Giannis Antetokounmpo,PF_73028-40199
0,Giannis Antetokounmpo,MIL,SAC,34.0,SF,11500.0,57.7,7.1,67.4,29.4,MAIN,Giannis Antetokounmpo,PF/SF,73028-40199,NaN,F,PF/SF/F,73028-40199:Giannis Antetokounmpo,SF_73028-40199
1,Stephen Curry,GSW,BOS,36.0,PG,10000.0,43.6,-1.4,53.2,0.0,MAIN,Stephen Curry,PG,73028-9524,G,NaN,PG/G,73028-9524:Stephen Curry,PG_73028-9524
2,Jayson Tatum,BOS,GSW,38.0,SF,9700.0,46.0,-0.6,55.8,0.2,MAIN,Jayson Tatum,SF/PF,73028-80808,NaN,F,SF/PF/F,73028-80808:Jayson Tatum,SF_73028-80808
2,Jayson Tatum,BOS,GSW,38.0,PF,9700.0,46.0,-0.6,55.8,0.2,MAIN,Jayson Tatum,SF/PF,73028-80808,NaN,F,SF/PF/F,73028-80808:Jayson Tatum,PF_73028-80808


In [89]:
df = pd.read_csv("FD_final_PlayerPool_ETR_"+str(today)+".csv", index_col=['Id', 'FD Position'], skipinitialspace=True)

legal_assignments = df.index # tuples of (name, pos)
name_set = df.index.unique(0) # a convenience

costs = df['FD Salary'].to_dict()
values = df['FD Points'].to_dict()

# set up LP
draft = pulp.LpVariable.dicts('selected', legal_assignments, cat='Binary')

prob = pulp.LpProblem('the draft', LpMaximize)

#obj
prob += pulp.lpSum([draft[n,p]*values[n,p] for (n,p) in legal_assignments])

#salary cap
prob += pulp.lpSum([draft[n,p]*costs[n,p] for (n, p) in legal_assignments]) <= 60000

# positions
prob += pulp.lpSum([draft[n, p] for (n, p) in legal_assignments if p == 'PG']) == 2
prob += pulp.lpSum([draft[n, p] for (n, p) in legal_assignments if p == 'SG']) == 2
prob += pulp.lpSum([draft[n, p] for (n, p) in legal_assignments if p == 'SF']) == 2
prob += pulp.lpSum([draft[n, p] for (n, p) in legal_assignments if p == 'PF']) == 2
prob += pulp.lpSum([draft[n, p] for (n, p) in legal_assignments if p == 'C']) == 1

# use each player at most only once
result = []
for name in name_set:
    prob += pulp.lpSum([draft[n, p] for (n, p) in legal_assignments if n == name]) <= 1

prob.solve()

for idx in draft:
    if draft[idx].varValue > 0:
        print(idx[0], idx[1])
        
        result.append([idx[0], idx[1]])

73028-40199 PF
73028-59358 C
73028-24247 PG
73028-16261 SG
73028-67026 SF
73028-23670 PG
73028-145314 PF
73028-84711 SG
73028-16604 SF


C:\Users\vchang\Anaconda3\lib\site-packages\pulp\pulp.py:1313: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [90]:
# new_list = result
# df = pd.DataFrame(new_list)
# test = df[[1,0]]
# test1 = test.transpose()
# test2 = test1.rename(columns=test1.iloc[0])
# test3 = test2.drop([1])
# df2 = test3[['PG', 'SG','SF','PF', 'C']]
# df2.to_csv("fd_result_" + str(today) + ".csv", index=None)

In [91]:
new_list = result
df = pd.DataFrame(new_list)
test = df[[1,0]]

# Create a lookup variable for [Position + Id]
test['lookup'] = test[1] + "_" + test[0]


# Merge PuLP output on [Position + Id] to pull in Player Name
df = pd.read_csv("FD_final_PlayerPool_ETR_"+str(today)+".csv")
upload = pd.merge(test,
                 df[['Player', 'Position + Id', 'Name + ID']],
                 left_on = 'lookup',
                 right_on = 'Position + Id')


# Drop all columns not used in upload
upload2 = upload.drop([0, 'lookup', 'Player', 'Position + Id'], axis = 1)


# Transpose data output from vertical to horizontal
upload3 = upload2.transpose()


# Rename columns to first row (position values)
upload4 = upload3.rename(columns=upload3.iloc[0])


# Drop duplicate first row
upload5 = upload4.drop([1])


# Re-order the columns to the Spositions
upload6 = upload5[['PG',
                   'SG',
                   'SF',
                   'PF',
                   'C']]


# Save as a csv
upload6.to_csv("fd_result_" + str(today) + ".csv", index=None)

# os.system("fd_result_" + str(today) + ".csv")

upload6

C:\Users\vchang\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,PG,PG,SG,SG,SF,SF,PF,PF,C
Name + ID,73028-24247:Fred VanVleet,73028-23670:Marcus Smart,73028-16261:Khris Middleton,73028-84711:Grayson Allen,73028-67026:Jaylen Brown,73028-16604:Justin Holiday,73028-40199:Giannis Antetokounmpo,73028-145314:Precious Achiuwa,73028-59358:Domantas Sabonis


In [92]:
upload6 = pd.read_csv(r'C:\Users\vchang\Documents\Fantasy Bball\DFS\PlayerPool Analysis\fd_result_' + str(today) + ".csv")
upload6.head()


upload = pd.read_csv(r'C:\Users\vchang\Documents\Fantasy Bball\DFS\Upload\FanDuel\FD_Upload_PlayerPool_' + str(today) + ".csv"
                     , error_bad_lines=False, engine ='python')


upload = upload[['entry_id', 'contest_id', 'contest_name']]
upload.head()

os.chdir(r'C:\Users\vchang\Documents\Fantasy Bball\DFS\Upload\FanDuel')
result = pd.concat([upload, upload6], axis=1)
test = result.rename(columns={"PG.1":"PG", "SG.1":"SG", "SF.1":"SF", "PF.1":"PF"})
result.to_csv("FD_Final_Upload_"+ time + str(today)+".csv", index=False)
os.system("FD_Final_Upload_"+ time + str(today)+".csv")

Skipping line 7: Expected 14 fields in line 7, saw 31
Skipping line 8: Expected 14 fields in line 8, saw 31
Skipping line 9: Expected 14 fields in line 9, saw 31
Skipping line 10: Expected 14 fields in line 10, saw 31
Skipping line 11: Expected 14 fields in line 11, saw 31
Skipping line 12: Expected 14 fields in line 12, saw 31
Skipping line 13: Expected 14 fields in line 13, saw 31
Skipping line 14: Expected 14 fields in line 14, saw 31
Skipping line 15: Expected 14 fields in line 15, saw 31
Skipping line 16: Expected 14 fields in line 16, saw 31
Skipping line 17: Expected 14 fields in line 17, saw 31
Skipping line 18: Expected 14 fields in line 18, saw 31
Skipping line 19: Expected 14 fields in line 19, saw 31
Skipping line 20: Expected 14 fields in line 20, saw 31
Skipping line 21: Expected 14 fields in line 21, saw 31
Skipping line 22: Expected 14 fields in line 22, saw 31
Skipping line 23: Expected 14 fields in line 23, saw 31
Skipping line 24: Expected 14 fields in line 24, saw 3

0

In [ ]:
test